In [1]:
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables
from model_builders.singleton_classifier import SingletonClassifierModelBuilder
from functools import reduce

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
embedding_indexes_file_path = 'helper_files/embedding/embedding_indexes.txt'
indexed_embedding_file_path = 'helper_files/embedding/indexed_embedding.txt'

word_vector, embedding_matrix, idx_by_word, word_by_idx = get_embedding_variables(embedding_indexes_file_path, indexed_embedding_file_path)

In [3]:
data = get_markable_dataframe("data/training/markables.csv", word_vector, idx_by_word)

data.head()

,id,text,is_pronoun,entity_type,is_proper_name,is_first_person,previous_words,next_words,is_singleton
0,1,[1],0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,[],"[2, 3, 4, 5, 6, 7, 8, 9, 10, 2]","[0.0, 1.0]"
1,2,"[4, 5]",0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,"[1, 2, 3]","[6, 7, 8, 9, 10, 2, 11, 12, 13, 14]","[0.0, 1.0]"
2,3,"[6, 7, 8]",0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]",1,0,"[1, 2, 3, 4, 5]","[9, 10, 2, 11, 12, 13, 14, 15, 16, 17]","[1.0, 0.0]"
3,4,[10],0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9]","[2, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[0.0, 1.0]"
4,5,"[12, 13]",0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,"[3, 4, 5, 6, 7, 8, 9, 10, 2, 11]","[14, 15, 16, 17, 18, 19, 4, 5, 20, 21]","[0.0, 1.0]"


In [4]:
max_text_length = 10
max_prev_words_length = 10
max_next_words_length = 10

data_text = pad_sequences(data.text, maxlen=max_text_length, padding='post')
data_previous_words = pad_sequences(data.previous_words.map(lambda seq: seq[(-1*max_prev_words_length):]), maxlen=max_prev_words_length, padding='pre')
data_next_words = pad_sequences(data.next_words.map(lambda seq: seq[:max_next_words_length]), maxlen=max_next_words_length, padding='post')
data_syntactic = data[['is_pronoun', 'entity_type', 'is_proper_name', 'is_first_person']]

data_syntactic = np.array(list(map(lambda p: reduce(lambda x,y: x + y, [i if type(i) is list else [i] for i in p]), data_syntactic.values)))
label = np.vstack(data.is_singleton)

# Build Model

## Words

In [5]:
words_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=False,
    embedding_matrix=embedding_matrix
)

In [6]:
words_model = words_model_builder.create_model()

In [7]:
words_model.fit([data_text], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 4s 731us/step - loss: 0.2993 - acc: 0.8591
Epoch 2/20
5032/5032 [==============================] - 2s 490us/step - loss: 0.1987 - acc: 0.8951
Epoch 3/20
5032/5032 [==============================] - 2s 463us/step - loss: 0.1655 - acc: 0.9231
Epoch 4/20
5032/5032 [==============================] - 3s 497us/step - loss: 0.1489 - acc: 0.9314
Epoch 5/20
5032/5032 [==============================] - 2s 459us/step - loss: 0.1339 - acc: 0.9388
Epoch 6/20
5032/5032 [==============================] - 2s 451us/step - loss: 0.1302 - acc: 0.9374
Epoch 7/20
5032/5032 [==============================] - 3s 506us/step - loss: 0.1401 - acc: 0.9394
Epoch 8/20
5032/5032 [==============================] - 3s 524us/step - loss: 0.1471 - acc: 0.9356
Epoch 9/20
5032/5032 [==============================] - 2s 490us/step - loss: 0.1195 - acc: 0.9436
Epoch 10/20
5032/5032 [==============================] - 3s 499us/step - loss: 0.1219 - acc: 0.9420
Epoch 11/

In [8]:
words_model.save('models/singleton_classifiers/words.model')

## Context

In [9]:
context_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    embedding_matrix=embedding_matrix
)

In [10]:
context_model = context_model_builder.create_model()

In [11]:
context_model.fit([data_previous_words, data_next_words], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 7s 1ms/step - loss: 0.5316 - acc: 0.8221
Epoch 2/20
5032/5032 [==============================] - 4s 793us/step - loss: 0.4127 - acc: 0.8374
Epoch 3/20
5032/5032 [==============================] - 4s 784us/step - loss: 0.3253 - acc: 0.8454
Epoch 4/20
5032/5032 [==============================] - 4s 798us/step - loss: 0.2788 - acc: 0.8720
Epoch 5/20
5032/5032 [==============================] - 4s 803us/step - loss: 0.2726 - acc: 0.8800
Epoch 6/20
5032/5032 [==============================] - 4s 763us/step - loss: 0.2105 - acc: 0.9163
Epoch 7/20
5032/5032 [==============================] - 4s 829us/step - loss: 0.1821 - acc: 0.9300
Epoch 8/20
5032/5032 [==============================] - 4s 811us/step - loss: 0.1554 - acc: 0.9428
Epoch 9/20
5032/5032 [==============================] - 4s 800us/step - loss: 0.1238 - acc: 0.9547
Epoch 10/20
5032/5032 [==============================] - 4s 774us/step - loss: 0.1135 - acc: 0.9610
Epoch 11/20

In [12]:
context_model.save('models/singleton_classifiers/context.model')

## Syntactic

In [13]:
syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    syntactic_features_num=data_syntactic.shape[1]
)

In [14]:
syntactic_model = syntactic_model_builder.create_model()

In [15]:
syntactic_model.fit([data_syntactic], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 2s 354us/step - loss: 0.4259 - acc: 0.8225
Epoch 2/20
5032/5032 [==============================] - 0s 97us/step - loss: 0.2623 - acc: 0.8416
Epoch 3/20
5032/5032 [==============================] - 1s 102us/step - loss: 0.2556 - acc: 0.8543
Epoch 4/20
5032/5032 [==============================] - 0s 94us/step - loss: 0.2528 - acc: 0.8525
Epoch 5/20
5032/5032 [==============================] - 0s 98us/step - loss: 0.2484 - acc: 0.8549
Epoch 6/20
5032/5032 [==============================] - 0s 94us/step - loss: 0.2525 - acc: 0.8583
Epoch 7/20
5032/5032 [==============================] - 0s 97us/step - loss: 0.2493 - acc: 0.8585
Epoch 8/20
5032/5032 [==============================] - 1s 120us/step - loss: 0.2475 - acc: 0.8591
Epoch 9/20
5032/5032 [==============================] - 1s 120us/step - loss: 0.2467 - acc: 0.8641
Epoch 10/20
5032/5032 [==============================] - 1s 103us/step - loss: 0.2470 - acc: 0.8593
Epoch 11/20
50

In [16]:
syntactic_model.save('models/singleton_classifiers/syntactic.model')

## Words + Context

In [17]:
words_context_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    embedding_matrix=embedding_matrix
)

In [18]:
words_context_model = words_context_model_builder.create_model()

In [19]:
words_context_model.fit([data_text, data_previous_words, data_next_words], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 9s 2ms/step - loss: 0.3877 - acc: 0.8398
Epoch 2/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1898 - acc: 0.9084
Epoch 3/20
5032/5032 [==============================] - 5s 978us/step - loss: 0.1417 - acc: 0.9404
Epoch 4/20
5032/5032 [==============================] - 5s 1ms/step - loss: 0.1227 - acc: 0.9485
Epoch 5/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0960 - acc: 0.9597
Epoch 6/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0807 - acc: 0.9676
Epoch 7/20
5032/5032 [==============================] - 5s 979us/step - loss: 0.0730 - acc: 0.9694
Epoch 8/20
5032/5032 [==============================] - 5s 1ms/step - loss: 0.0537 - acc: 0.9760
Epoch 9/20
5032/5032 [==============================] - 5s 980us/step - loss: 0.0495 - acc: 0.9787
Epoch 10/20
5032/5032 [==============================] - 5s 1ms/step - loss: 0.0478 - acc: 0.9803
Epoch 11/20
5032/5032 [

In [20]:
words_context_model.save('models/singleton_classifiers/words_context.model')

## Words + Syntactic

In [21]:
words_syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=data_syntactic.shape[1]
)

In [22]:
words_syntactic_model = words_syntactic_model_builder.create_model()

In [23]:
words_syntactic_model.fit([data_text, data_syntactic], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 5s 974us/step - loss: 0.3464 - acc: 0.8359
Epoch 2/20
5032/5032 [==============================] - 2s 445us/step - loss: 0.1945 - acc: 0.8824
Epoch 3/20
5032/5032 [==============================] - 2s 420us/step - loss: 0.1577 - acc: 0.9132
Epoch 4/20
5032/5032 [==============================] - 2s 416us/step - loss: 0.1378 - acc: 0.9281
Epoch 5/20
5032/5032 [==============================] - 2s 418us/step - loss: 0.1235 - acc: 0.9368
Epoch 6/20
5032/5032 [==============================] - 2s 419us/step - loss: 0.1106 - acc: 0.9408
Epoch 7/20
5032/5032 [==============================] - 2s 409us/step - loss: 0.1096 - acc: 0.9436
Epoch 8/20
5032/5032 [==============================] - 2s 429us/step - loss: 0.1076 - acc: 0.9465
Epoch 9/20
5032/5032 [==============================] - 2s 415us/step - loss: 0.1060 - acc: 0.9438
Epoch 10/20
5032/5032 [==============================] - 2s 404us/step - loss: 0.0993 - acc: 0.9483
Epoch 11/

In [24]:
words_syntactic_model.save('models/singleton_classifiers/words_syntactic.model')

## Context + Syntactic

In [25]:
context_syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=data_syntactic.shape[1]
)

In [26]:
context_syntactic_model = context_syntactic_model_builder.create_model()

In [27]:
context_syntactic_model.fit([data_previous_words, data_next_words, data_syntactic], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.4985 - acc: 0.8221
Epoch 2/20
5032/5032 [==============================] - 3s 689us/step - loss: 0.2982 - acc: 0.8567
Epoch 3/20
5032/5032 [==============================] - 4s 697us/step - loss: 0.2295 - acc: 0.8923
Epoch 4/20
5032/5032 [==============================] - 3s 694us/step - loss: 0.2019 - acc: 0.9122
Epoch 5/20
5032/5032 [==============================] - 4s 713us/step - loss: 0.1638 - acc: 0.9328
Epoch 6/20
5032/5032 [==============================] - 3s 690us/step - loss: 0.1496 - acc: 0.9422
Epoch 7/20
5032/5032 [==============================] - 3s 694us/step - loss: 0.1195 - acc: 0.9535
Epoch 8/20
5032/5032 [==============================] - 4s 777us/step - loss: 0.1162 - acc: 0.9535
Epoch 9/20
5032/5032 [==============================] - 4s 769us/step - loss: 0.0915 - acc: 0.9688
Epoch 10/20
5032/5032 [==============================] - 4s 767us/step - loss: 0.1005 - acc: 0.9668
Epoch 11/20

In [28]:
context_syntactic_model.save('models/singleton_classifiers/context_syntactic.model')

## Words + Context + Syntactic

In [29]:
words_context_syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=data_syntactic.shape[1]
)

In [30]:
words_context_syntactic_model = words_context_syntactic_model_builder.create_model()

In [31]:
words_context_syntactic_model.fit([data_text, data_previous_words, data_next_words, data_syntactic], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 9s 2ms/step - loss: 0.4045 - acc: 0.8309
Epoch 2/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.2114 - acc: 0.8961
Epoch 3/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1602 - acc: 0.9243
Epoch 4/20
5032/5032 [==============================] - 7s 1ms/step - loss: 0.1322 - acc: 0.9416
Epoch 5/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1010 - acc: 0.9601
Epoch 6/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0824 - acc: 0.9694
Epoch 7/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0687 - acc: 0.9722
Epoch 8/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0536 - acc: 0.9795
Epoch 9/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0489 - acc: 0.9817
Epoch 10/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0390 - acc: 0.9843
Epoch 11/20
5032/5032 [======

In [32]:
words_context_syntactic_model.save('models/singleton_classifiers/words_context_syntactic.model')